In [14]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [15]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [16]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

193

In [17]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

In [18]:
len(embedding.embed_query("소득세법"))

4096

In [23]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
index_name = "tax-index-upstage"
database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name
)

# 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets

# 업로드한 벡터db를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 5.95 s
Wall time: 50.1 s


# 2. 답변 생성을 위한 Retrieval

In [24]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

In [8]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [9]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 3. 답변 생성

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4.1-nano")

In [11]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [12]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [13]:
ai_message = llm.invoke(prompt)

In [14]:
print(ai_message.content)

**연봉 5천만원 직장인의 소득세 계산 (2023년 기준)**

### 1. **근로소득금액**  
- 연봉 5,000만원 = **5,000만원** (근로소득 전액)

---

### 2. **근로소득 공제**  
- **5,000만원 이하** 구간 적용:  
  \[
  1,200만원 + (5,000만원 - 1,200만원) \times 15\% = 1,200만원 + 540만원 = **1,740만원**
  \]  
- **과세표준**:  
  \[
  5,000만원 - 1,740만원 = **3,260만원**
  \]

---

### 3. **종합소득세 계산**  
- **세율 구간**: 1,200만원 초과 ~ 4,600만원 이하  
  - 1,200만원 × 6% = **72만원**  
  - (3,260만원 - 1,200만원) × 15% = **2,060만원 × 15% = 309만원**  
  - **산출세액**: 72만원 + 309만원 = **381만원**  
- **누진공제**: 108만원  
  \[
  381만원 - 108만원 = **273만원**
  \]

---

### 4. **지방소득세**  
- 종합소득세의 10%:  
  \[
  273만원 × 10\% = **27.3만원**
  \]

---

### 5. **총 납부 세액**  
\[
273만원 (종합소득세) + 27.3만원 (지방소득세) = **273만원 + 27.3만원 = 300.3만원**
\]

---

### **최종 답변**  
- **예상 소득세**: 약 **300만원**  
- **참고**: 실제 세금은 의료비, 보험료, 신용카드 소득공제 등 추가 공제에 따라 달라질 수 있습니다. 정확한 계산을 위해선 [홈택스](https://www.hometax.go.kr)에서 연말정산 시뮬레이션을 활용하시기 바랍니다.


# 4. langchain 답변 생성
- ch9.07_LangChain과 vectorDatabase을 활용한 RAG구현(UpstageEmbedding) 참조

In [15]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model = "gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")

embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

# 2. # 업로드한 벡터db를 가져올 때
vectorstore = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])

In [16]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'연봉 5천만원의 소득세는 근로소득 공제, 누진세율, 세액공제 등을 적용해야 정확히 계산 가능하며, 약 180~220만원(지방소득세 제외) 정도로 추정할 수 있습니다. 정확한 금액은 추가 정보가 필요합니다.  \n\n(참고: 2023년 기준 근로소득세율 및 공제제도 적용)'

# 5. 성능

In [7]:
# 업로드한 백터 db를 가져오기
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

load_dotenv()
pc = Pinecone(
    api_key=os.getenv('PINECONE_API_KEY')
)

embedding = UpstageEmbeddings(model = "solar-embedding-1-large-passage")

vectorstore = PineconeVectorStore(
    embedding = embedding, 
    index_name = "tax-index-upstage"
)

In [11]:
retriever = vectorstore.as_retriever(
    # search_type = "similarity", "mmr", 
    # search_kwargs = {"k"L4}
)

retrieved_docs = retriever.invoke("연봉 5천만원인 직장인의 소득세는 얼마인가요?")
for doc in retrieved_docs:
    print(doc.page_content)
    print("---------------------------")

2. 2명인 경우: 연 55만원

3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액

② 삭제<2017. 12. 19.>

③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>

1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원

2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원

3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원

④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>

[본조신설 2014. 1. 1.]

[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]



제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 한다. <개정 2014. 12. 23., 2015. 5. 13., 2016. 12. 20., 2022. 12. 31.>

1. 제146조제2항에 따라 소득세가 원천징수되지 아니한 퇴직소득 등 과세가 이연된 소득

2. 연금계좌에서 다른 연금계좌로 계약을 이전함으로써 납입되

In [13]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = vectorstore.similarity_search(query, k = 3)
for doc in retrieved_docs:
    print(doc)
    print("-----------------")

page_content='2. 2명인 경우: 연 55만원

3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액

② 삭제<2017. 12. 19.>

③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>

1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원

2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원

3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원

④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>

[본조신설 2014. 1. 1.]

[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]



제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 한다. <개정 2014. 12. 23., 2015. 5. 13., 2016. 12. 20., 2022. 12. 31.>

1. 제146조제2항에 따라 소득세가 원천징수되지 아니한 퇴직소득 등 과세가 이연된 소득

2. 연금계좌에서 다른 연금계좌로 

In [ ]:
# query에 가장 유사도가 높은 55조는 이미지로 되어 있어 chunk에 미포함
=> 이미지를 table로 변경 (ch9.10예제)